# Classification Signs

In [2]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
#from tensorflow import keras

from matplotlib import patches

import keras
from tensorflow.keras.models import load_model

from keras import Sequential                                                                      
from keras.models import Model                                                                    
from keras.callbacks import EarlyStopping                                                         
from keras.layers import Dense, Input, Convolution2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout, Softmax, Activation
from keras import optimizers                                                                      
from tensorflow.keras.models import load_model                                                    

import matplotlib.pyplot as plt
import os as os

import json
import PIL as PIL
from PIL import Image
from PIL import ImageOps
import random 

## Read in GTSRB

In [3]:
path_data = '/home/cpf/Desktop/GTSRB/Final_Training/Images'

In [4]:
def image_numpy_reshape(path, x_size = 50, y_size = 50):
    """ Input: path to image, x_size, y_size of the output
        Output: numpy array"""
    
    big_enough = False
        
    image_data = PIL.Image.open(path)
    xs, ys, cs = np.asarray(image_data).shape
    if (xs > x_size + 15) and (y_size+15 > 35):
        big_enough = True
        image_data.thumbnail((x_size+10, y_size+10), Image.ANTIALIAS)
        image_data = ImageOps.fit(image_data, (x_size, y_size), Image.ANTIALIAS)
        image_data = np.asarray(image_data)
    
    return big_enough, image_data

In [5]:
number_elements = 1300

image_array = np.zeros((number_elements, 50, 50, 3))
label_array = np.zeros(number_elements) - 1

t_counter = 0

#for ending in tqdm(['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008']):
for ending in tqdm(['00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008']):
    data_files = os.listdir(path_data + '/' + ending)
    counter = 0
    for file in data_files:
        if ('.ppm' in file) and (counter < 250) and (t_counter < number_elements):
            path = path_data + '/' + ending + '/' + file
            big_enough, image_data = image_numpy_reshape(path)
            if big_enough == True:
                image_array[t_counter] = image_data/255
                label_array[t_counter] = float(ending[-1:]) - 1 
                counter += 1
                t_counter += 1
                if t_counter == number_elements - 1:
                    print("all_in")

100%|██████████| 8/8 [00:02<00:00,  3.75it/s]

all_in


In [6]:
for i in range(-1,9):
    print(i, np.count_nonzero(label_array == i))

-1 0
0 250
1 250
2 164
3 232
4 148
5 32
6 147
7 77
8 0


## Shuffle and Split the data

In [7]:
p = np.random.permutation(len(image_array))
image_array_shuffled, label_array_shuffled = image_array[p], label_array[p]

split = 1300

training_images = image_array_shuffled[:split]
training_labels = label_array_shuffled[:split]

test_images = image_array_shuffled[split:]
test_labels = label_array_shuffled[split:]

## Read in the Test Data

In [8]:
# we test on a set with 300 images:
path_data_test = '/home/cpf/Desktop/GTSRB-Online-Test-sort'

number_elements = 250

image_array_test = np.zeros((number_elements, 50, 50, 3))
label_array_test = np.zeros(number_elements) - 1 - 1

t_counter = 0

#for ending in tqdm(['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008']):
for ending in tqdm(['00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008']):
    data_files_test = os.listdir(path_data_test + '/' + ending)
    counter = 0
    for file in data_files_test:
        if ('.ppm' in file) and (counter < 40) and (t_counter < number_elements):
            path = path_data_test + '/' + ending + '/' + file
            big_enough, image_data = image_numpy_reshape(path)
            if big_enough == True:
                image_array_test[t_counter] = image_data/255
                label_array_test[t_counter] = float(ending[-1:]) - 1
                counter += 1
                t_counter += 1
                if t_counter == number_elements - 1:
                    print("all_in")

100%|██████████| 8/8 [00:00<00:00, 17.24it/s]

all_in


## Set up the Model

In [9]:
model = Sequential()                                                 
model.add(Convolution2D(32, (3, 3),activation='relu', input_shape=(50,50,3)))   
model.add(MaxPooling2D(pool_size=(3, 3)))
#model.add(AveragePooling2D(pool_size = (2, 2)))
model.add(Convolution2D(32, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(AveragePooling2D(pool_size = (2, 2)))
#model.add(Convolution2D(15, (3,3),activation='relu'))
#model.add(AveragePooling2D(pool_size = (2, 2)))
model.add(Flatten()),   
#model.add(Dense(10, activation='relu'))     
model.add(Dense(8, activation='softmax'))     
model.build()                                                        

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 12552     
Total params: 22,696
Trainable params: 22,696
Non-trainable params: 0
__________________________________________________

## Train the Model

In [11]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=5)

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
model.fit(training_images, training_labels, epochs=100,
                                            validation_split=0.1, 
                                            shuffle=True, 
                                            verbose=1,
                                            callbacks=[es])


Train on 1170 samples, validate on 130 samples
Epoch 1/100
1170/1170 [==============================] - 2s 2ms/step - loss: 2.0157 - accuracy: 0.3983 - val_loss: 1.9835 - val_accuracy: 0.0000e+00
Epoch 2/100
1170/1170 [==============================] - 2s 1ms/step - loss: 1.9666 - accuracy: 0.2650 - val_loss: 1.9160 - val_accuracy: 0.6846
Epoch 3/100
1170/1170 [==============================] - 2s 1ms/step - loss: 1.9267 - accuracy: 0.3402 - val_loss: 1.8604 - val_accuracy: 0.4846
Epoch 4/100
1170/1170 [==============================] - 2s 1ms/step - loss: 1.8341 - accuracy: 0.2333 - val_loss: 1.7795 - val_accuracy: 0.2154
Epoch 5/100
1170/1170 [==============================] - 2s 1ms/step - loss: 1.7126 - accuracy: 0.1957 - val_loss: 1.7018 - val_accuracy: 0.1769
Epoch 6/100
1170/1170 [==============================] - 2s 1ms/step - loss: 1.6297 - accuracy: 0.2085 - val_loss: 1.5974 - val_accuracy: 0.2385
Epoch 7/100
1170/1170 [==============================] - 2s 1ms/step - loss: 1

## Evaluate Model

In [16]:
predictions = model.predict(image_array_test)

In [17]:
correct = np.argmax(predictions, axis=1) == label_array_test 
accuracy = np.sum(correct) / correct.size
print('\nTest accuracy:', accuracy)


Test accuracy: 0.936


In [18]:
model.save('class_models/classification_model_' + str(accuracy) + '.h5')

***

In [ ]:
image_number = 151
plt.imshow(image_array_test[image_number])
plt.show()
print(predictions[image_number])
print(label_array_test[image_number])

plt.plot(range(9), predictions[image_number], "o")
plt.plot(label_array_test[image_number], [0], "o", color="red")
plt.grid(True)
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('\nTest accuracy:', test_acc)